# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
import cartopy.crs as ccrs
import geoviews as gv 


# Import API key
from api_keys import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bredasdorp,-34.5322,20.0403,17.44,68,51,3.70,ZA,1679941021
1,1,aklavik,68.2191,-135.0107,-1.19,78,100,1.77,CA,1679941369
2,2,airai,-8.9266,125.4092,15.04,94,98,0.96,TL,1679941525
3,3,tasiilaq,65.6145,-37.6368,-9.91,79,7,1.75,GL,1679941363
4,4,cape town,-33.9258,18.4232,22.99,55,0,3.13,ZA,1679941203


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "Country"
)

# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [44]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather= city_data_df[(city_data_df["Humidity"]<50) & (city_data_df["Humidity"]>20) &(city_data_df[ "Wind Speed"]<10)
                        & (city_data_df["Max Temp"]> 10) & (city_data_df["Max Temp"]<50)& (city_data_df["Cloudiness"] <10)]
# YOUR CODE HERE
ideal_weather= ideal_weather.dropna(how="any")
# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
102,102,melo,-32.3667,-54.1833,26.50,32,0,2.69,UY,1679941919
104,104,mar del plata,-38.0023,-57.5575,25.53,47,0,8.23,AR,1679941343
129,129,joacaba,-27.1781,-51.5047,28.97,27,0,1.31,BR,1679941928
132,132,colares,38.7992,-9.4469,21.55,44,0,3.60,PT,1679941929
134,134,baghmara,23.7950,86.2085,26.22,27,9,2.42,IN,1679941930
136,136,chuy,-33.6971,-53.4616,23.74,49,0,2.77,UY,1679941396
158,158,sakakah,29.9697,40.2064,17.76,29,1,3.80,SA,1679941763
218,218,rocha,-34.4833,-54.3333,25.01,41,0,2.66,UY,1679941718
229,229,tecoanapa,16.5167,-98.7500,29.16,49,0,4.49,MX,1679941971
237,237,lamar,33.6668,-95.5836,18.83,42,0,2.57,US,1679941973


### Step 3: Create a new DataFrame called `hotel_df`.

In [45]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= ideal_weather[["City", "Country","Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel name"]=""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel name
102,melo,UY,-32.3667,-54.1833,32,
104,mar del plata,AR,-38.0023,-57.5575,47,
129,joacaba,BR,-27.1781,-51.5047,27,
132,colares,PT,38.7992,-9.4469,44,
134,baghmara,IN,23.7950,86.2085,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [46]:
# Set parameters to search for a hotel
radius = 10000
limit=10
categories= "accomodation.hotel"
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
melo - nearest hotel: No hotel found
mar del plata - nearest hotel: No hotel found
joacaba - nearest hotel: No hotel found
colares - nearest hotel: No hotel found
baghmara - nearest hotel: No hotel found
chuy - nearest hotel: No hotel found
sakakah - nearest hotel: No hotel found
rocha - nearest hotel: No hotel found
tecoanapa - nearest hotel: No hotel found
lamar - nearest hotel: No hotel found
san rafael - nearest hotel: No hotel found
nandurbar - nearest hotel: No hotel found
korla - nearest hotel: No hotel found
oudtshoorn - nearest hotel: No hotel found
san juan - nearest hotel: No hotel found
dumas - nearest hotel: No hotel found
pedernales - nearest hotel: No hotel found
progreso - nearest hotel: No hotel found
rengo - nearest hotel: No hotel found
catalina - nearest hotel: No hotel found
palanpur - nearest hotel: No hotel found
ajdabiya - nearest hotel: No hotel found
necochea - nearest hotel: No hotel found
punta alta - nearest hotel: No hotel found
takha

,City,Country,Lat,Lng,Humidity,Hotel name,Hotel Name
102,melo,UY,-32.3667,-54.1833,32,,No hotel found
104,mar del plata,AR,-38.0023,-57.5575,47,,No hotel found
129,joacaba,BR,-27.1781,-51.5047,27,,No hotel found
132,colares,PT,38.7992,-9.4469,44,,No hotel found
134,baghmara,IN,23.7950,86.2085,27,,No hotel found
136,chuy,UY,-33.6971,-53.4616,49,,No hotel found
158,sakakah,SA,29.9697,40.2064,29,,No hotel found
218,rocha,UY,-34.4833,-54.3333,41,,No hotel found
229,tecoanapa,MX,16.5167,-98.7500,49,,No hotel found
237,lamar,US,33.6668,-95.5836,42,,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [47]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "Country",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,Hotel Name)